#### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.color import rgb2gray
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.filters import threshold_otsu


In [ ]:
original_image = cv2.imread('../../dataset/colourcropped/17.jpg')

### Sobel Edge detection

In [ ]:
image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

# Convert image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Apply Sobel edge detection
sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
gradient_magnitude = cv2.magnitude(sobel_x, sobel_y)

# Invert the gradient magnitude
inverted_gradient = 255 - gradient_magnitude

# Display the original image
#make the figure larger
plt.figure(figsize=(20, 15))
plt.subplot(131), plt.imshow(image), plt.title('Original Image')

# Display the Sobel edges with white background
plt.subplot(132), plt.imshow(inverted_gradient, cmap='gray'), plt.title('Sobel Edges (Black on White)')
plt.show()

#save inverted_gradient as a image
cv2.imwrite('../report/edge_detection/sobel.jpg', inverted_gradient)


### Canny 

In [ ]:
img = cv2.resize(original_image,(int(original_image.shape[1]/2), int(original_image.shape[0]/2)))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#apply gausssian blur
img = cv2.GaussianBlur(img, (5,5), 0)
canny = cv2.Canny(img,threshold1=190, threshold2=200)
inverted_canny = 255 - canny
#display canny

# display inverted canny
plt.figure(figsize=(20, 15))
plt.subplot(131), plt.imshow(image), plt.title('Original Image')
plt.subplot(132), plt.imshow(inverted_canny, cmap='gray'), plt.title('Canny Edges (Black on White)')
plt.show()


# cv2.imwrite('../report/edge_detection/canny.jpg', inverted_canny)

# cv2.imshow('Frame View', inverted_canny)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
from PIL import Image
import numpy as np

# change name to create colour hints
def random_sample_rgb_same_shape(image, num_samples=10, patch_size=(20, 20)):
    height, width, channels = image.shape
    
    # Ensure that the image is large enough for sampling
    if height < patch_size[0] or width < patch_size[1]:
        raise ValueError("Image is too small for the specified patch size.")
    
    # Initialize the matrix to store samples
    samples_matrix = np.zeros((height, width, 3), dtype=image.dtype)
    
    for _ in range(num_samples):
        start_row = np.random.randint(0, height - patch_size[0] + 1)
        start_col = np.random.randint(int(width * 0.25), int(width * 0.75) - patch_size[1] + 1)
        
        patch = image[start_row:start_row+patch_size[0], start_col:start_col+patch_size[1], :]
        # apply guassian blur to the patch
        patch = cv2.GaussianBlur(patch, (5,5), 0)
        # make the patch values be the average of the patch
        patch = np.mean(patch, axis=(0, 1)).astype(np.float32)

        samples_matrix[start_row:start_row+patch_size[0], start_col:start_col+patch_size[1], :] = patch
    
    return samples_matrix

# Load the image
image = Image.open('/dcs/large/u2146727/kaggle/data/val/1301041.png')
# split the image in half and display the two halves (left and right)
left, right = image.crop((0, 0, 512, 512)), image.crop((512, 0, 1024, 512))
image = np.array(left)

# Randomly sample 10 patches from the image
samples = random_sample_rgb_same_shape(image, num_samples=10)
# Display the sampled patches togethe

# Display the original and sampled patches
plt.figure(figsize=(20, 15))
plt.subplot(121), plt.imshow(left), plt.title('Original Image')
plt.subplot(122), plt.imshow(samples), plt.title('Sampled Patches')
plt.show()

# stack the images together
# mask = np.any(samples > 0, axis=2)
# gray_original = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
# # # Convert greyscale image to a 3-channel image for compatibility
# greyscale_image_colored = cv2.cvtColor(gray_original, cv2.COLOR_GRAY2BGR)

# Create a mask based on non-black regions in the samples_matrix
mask = np.any(samples > 0, axis=2)

# Ensure greyscale_image_colored and samples_matrix have the same number of channels
#greyscale_image_colored = cv2.cvtColor(greyscale_image_colored, cv2.COLOR_BGR2RGB)

# Apply the mask to combine the images
result_image = np.where(mask[:, :, np.newaxis], samples, right)

# Display the result
plt.figure(figsize=(8, 6))
plt.imshow(result_image)
plt.show()




In [ ]:
from PIL import Image
import numpy as np

# make the function similar to the on above but this time the patches are a random colour
def random_colour_rgb_same_shape(image, num_samples=8, patch_size=(20, 20)):
    height, width, channels = image.shape
    
    # Ensure that the image is large enough for sampling
    if height < patch_size[0] or width < patch_size[1]:
        raise ValueError("Image is too small for the specified patch size.")
    
    # Initialize the matrix to store samples
    samples_matrix = np.zeros((height, width, 3), dtype=image.dtype)
    
    for _ in range(num_samples):
        start_row = np.random.randint(0, height - patch_size[0] + 1)
        start_col = np.random.randint(int(width * 0.25), int(width * 0.75) - patch_size[1] + 1)
        # set the wholr patch to a same random colour
        patch = np.random.randint(0, 256, 3).astype(np.uint8)
        samples_matrix[start_row:start_row+patch_size[0], start_col:start_col+patch_size[1], :] = patch
    
    return samples_matrix

image = Image.open('/dcs/large/u2146727/kaggle/data/val/2149049.png')
# split the image in half and display the two halves (left and right)
left, right = image.crop((0, 0, 512, 512)), image.crop((512, 0, 1024, 512))
image = np.array(left)

# Randomly sample 10 patches from the image
samples = random_colour_rgb_same_shape(image, num_samples=5)
mask = np.any(samples > 0, axis=2)


# Apply the mask to combine the images
result_image = np.where(mask[:, :, np.newaxis], samples, right)

# Display the result
plt.figure(figsize=(8, 6))
plt.imshow(result_image)
plt.show()

result_image = Image.fromarray(result_image)
result_image.save('/dcs/21/u2146727/cs310/dataset/val_random/2149049.png')


In [ ]:
# make a function similar to the one above but this time it takes coordinates and a colour as input
def colour_hint_manual(current_mask, coordinates, colour):
    # make the 20x20 patch to the colour given
    # and add it to the current mask
    patch = np.array(colour).astype(np.uint8)
    for coord in coordinates:
        start_row = coord[0]
        start_col = coord[1]
        current_mask[start_row:start_row+20, start_col:start_col+20, :] = patch
    
    return current_mask

image = Image.open('/dcs/large/u2146727/kaggle/data/val100/324109.png')
# split the image in half and display the two halves (left and right)
left, right = image.crop((0, 0, 512, 512)), image.crop((512, 0, 1024, 512))
image = np.array(left)

height, width, channels = image.shape
samples = np.zeros((height, width, 3), dtype=image.dtype)
samples = colour_hint_manual(samples, [(60, 256)], [71, 49, 49])
samples = colour_hint_manual(samples, [(200, 180)], [71, 49, 49])
# samples = colour_hint_manual(samples, [(400, 220)], [225, 0, 0])
#samples = colour_hint_manual(samples, [(250, 300)], [230, 232, 220])
samples = colour_hint_manual(samples, [(380, 370)], [222, 18, 18])
samples = colour_hint_manual(samples, [(400, 250)], [222, 18, 18])
samples = colour_hint_manual(samples, [(400, 150)], [222, 18, 18])
samples = colour_hint_manual(samples, [(170, 260)], [235, 195, 96])


mask = np.any(samples > 0, axis=2)
result_image = np.where(mask[:, :, np.newaxis], samples, right)

# Display the result
plt.figure(figsize=(8, 6))
plt.imshow(result_image)
plt.show()
# save result image under the same name as the original image

# save the image
result_image = Image.fromarray(result_image)
result_image.save('/dcs/large/u2146727/kaggle/data/val_manual/324109.png')
